In [1]:
### WEB TRACKING ASSIGNMENT III 
print("**********************************")
print("  WEB SCRAPPING ASSIGNMENT III \n")
print("  OLUWAJOBA FATOLA  \n  Flip Robo Technology Internship")
print("**********************************\n")

**********************************
  WEB SCRAPPING ASSIGNMENT III 

  OLUWAJOBA FATOLA  
  Flip Robo Technology Internship
**********************************



In [2]:
## Install Selenium Package 
! pip install selenium

In [149]:
# Import all necessary library 
import selenium
from selenium.webdriver.common.by import By
from IPython.display import display
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from selenium import webdriver
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
import requests
import time
import re


In [168]:
# Configuring Selenium to use Chrome browser
driver = webdriver.Chrome(r"C:\Users\JOBA\Downloads\chromedriver_win32\chromedriver.exe")

In [169]:
# Connecting to the driver 
driver = webdriver.Chrome(r"chromedriver.exe")

# Q1. Write a python program which searches all the product under a particular product from www.amazon.in. The product to be searched will be taken as input from user. For e.g. If user input is ‘guitar’. Then search for guitars.

In [47]:
try:
    # Take user input for the product to search
    product = input("Enter the product to search: ")

    # Format the product for the Amazon search URL
    search_term = product.replace(" ", "+")

    # Construct the search URL
    search_url = f"https://www.amazon.in/s?k={search_term}"

    # Navigate to the search results page
    driver.get(search_url)

    # Wait for the product elements to be present on the page
    wait = WebDriverWait(driver, 20)
    product_elements = wait.until(EC.presence_of_all_elements_located((By.XPATH, "//div[@data-component-type='s-search-result']")))

    if len(product_elements) > 0:
        # Initialize a list to store the product names
        product_names = []

        for element in product_elements:
            try:
                # Find the product name within each product element
                name_element = element.find_element(By.XPATH, ".//span[@class='a-size-base-plus a-color-base a-text-normal']")
                name = name_element.text
                product_names.append(name)
            except NoSuchElementException:
                product_names.append("-")

        # Create a DataFrame with the scraped product names
        df = pd.DataFrame(product_names, columns=['Product Name'])

        # Display the DataFrame
        # display(df)
        
        # Display the full DataFrame
        with pd.option_context('display.max_rows', None, 'display.max_columns', None):
            display(df)

    else:
        print("No products found.")

except Exception as e:
    print("An error occurred:", e)


Enter the product to search: guitar


,Product Name
0,"Yamaha F280 Acoustic Rosewood Guitar (Natural, Beige)"
1,"Yamaha APX600 NA Thin Body Acoustic-Electric Guitar, Natural"
2,"YAMAHA F310-TBS Right Handed Spruce, Back/Side/Rib, Meranti Acoustic Guitar (Tobacco Sunburst, 6-Strings)"
3,Intern 38C Ebony Wood Cutaway Design Acoustic Guitar with Picks And Carry Bag (Black)
4,"YAMAHA F310, 6-Strings Rose Wood Acoustic Guitar, Natural"
5,"Yamaha F280 Acoustic Guitar, Tobacco Brown Sunburst"
6,YAMAHA FS80C NATURAL CONCERT CUTAWAY GUITAR
7,Intern INT-38C Acoustic Guitar Kit (Purple)
8,"Yamaha F280 Acoustic Guitar, Natural With Bajaao Gig Bag"
9,Intern Int-38c Sunburst Linden Wood Acoustic Guitar Kit with Carry Bag and Picks


# Q2. 
In the above question, now scrape the following details of each product listed in first 3 pages of your search results and save it in a data frame and csv. In case if any product has less than 3 pages in search results then scrape all the products available under that product name. Details to be scraped are: "Brand Name", "Name of the Product", "Price", "Return/Exchange", "Expected Delivery", "Availability" and “Product URL”. In case, if any of the details are missing for any of the product then replace it by “-“.

In [32]:
driver.get("https://www.amazon.in/")

In [33]:
search_input = driver.find_element(By.XPATH, "//input[@id='twotabsearchtextbox']")
search_input.send_keys('guitar')
search_button = driver.find_element(By.XPATH, "//input[@id='nav-search-submit-button']")
search_button.click()
time.sleep(3)

In [34]:
# Scrap all product URLs
product_urls = []
start = 0
end = 3
for page in range(start, end):  # For loop for scraping 3 pages
    urls = driver.find_elements(By.XPATH, '//a[@class="a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal"]')
    for url in urls:
        product_urls.append(url.get_attribute("href"))
    next_button = driver.find_element(By.XPATH, '//a[@class="s-pagination-item s-pagination-next s-pagination-button s-pagination-separator"]')
    next_button.click()
    time.sleep(2)

In [35]:
len(product_urls)

154

In [37]:
# Define lists to store the product details
Brand = []
Name = []
Price = []
Return = []
Availability = []
product_details = []

# Loop through each product URL and scrape the details
for url in product_urls:
    driver.get(url)
    time.sleep(2)

    # Scrape the brand name
    try:
        brand = driver.find_element(By.XPATH, '//*[@id="productOverview_feature_div"]/div/table/tbody/tr[1]/td[2]/span').text
    except NoSuchElementException:
        brand = '-'
    Brand.append(brand)

    # Scrape the product name
    try:
        name = driver.find_element(By.XPATH, '//h1[@class="a-size-large a-spacing-none"]').text
    except NoSuchElementException:
        name = '-'
    Name.append(name)

    # Scrape the price
    try:
        price = driver.find_element(By.XPATH, '//*[@id="corePrice_feature_div"]/div/span[1]/span[2]').text
    except NoSuchElementException:
        price = '-'
    Price.append(price)

    # Scrape the return/exchange info
    try:
        return_info = driver.find_element(By.XPATH, '//*[@id="RETURNS_POLICY"]/span/div[2]/span').text
    except NoSuchElementException:
        return_info = '-'
    Return.append(return_info)

    # Scrape the availability
    try:
        availability = driver.find_element(By.XPATH, '//span[@class="a-size-medium a-color-success"]').text
    except NoSuchElementException:
        availability = '-'
    Availability.append(availability)

    # Append the product details to the list
    product_details.append([brand, name, price, return_info, availability, url])

# Create a DataFrame with the scraped product details
df = pd.DataFrame(product_details, columns=["Brand Name", "Product Name", "Price", "Return/Exchange", "Availability", "Product URL"])

# Display the full DataFrame without truncation
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
display(df)

# Save the DataFrame as a CSV file
df.to_csv("product_details.csv", index=False)

,Brand Name,Product Name,Price,Return/Exchange,Availability,Product URL
0,Kadence,Kadence Frontier Jumbo Semi Acoustic Guitar Wi...,"₹6,899\n00",7 days Replacement,In stock,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
1,Kadence,Kadence Frontier guitar with Online Guitar lea...,"₹4,999\n00",7 days Replacement,In stock,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
2,VAULT,Vault Traveller 34 Inch Acoustic Guitar With T...,"₹4,084\n00",7 days Replacement,In stock,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
3,VAULT,Vault DA40 41 Inch Premium Solid Spruce-Top Dr...,"₹6,934\n00",7 days Replacement,In stock,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
4,JUAREZ,"Juârez Acoustic Guitar, 38 Inch Cutaway, 038C ...","₹1,994\n00",7 days Replacement,In stock,https://www.amazon.in/Juarez-Acoustic-Cutaway-...
5,Intern,"Intern INT-38C Right hand Acoustic Guitar Kit,...","₹1,994\n00",7 days Replacement,In stock,https://www.amazon.in/Intern-INT-38C-Acoustic-...
6,JUAREZ,"Juarez Lindenwood Acoustic Guitar Kit, 38 Inch...","₹1,994\n00",7 days Replacement,In stock,https://www.amazon.in/JUAREZ-JRZ38C-Acoustic-S...
7,YAMAHA,"Yamaha F280 Acoustic Rosewood Guitar (Natural,...","₹7,270\n00",7 days Replacement,Available to ship in 1-2 days,https://www.amazon.in/Yamaha-F280-Acoustic-Gui...
8,JUAREZ,"Juarez Lindenwood Acoustic Guitar Kit, 38 Inch...","₹1,994\n00",7 days Replacement,In stock,https://www.amazon.in/JUAREZ-JRZ38C-Acoustic-S...
9,JUAREZ,"Juarez Lindenwood Acoustic Guitar, 38 Inches C...","₹2,640\n00",7 days Replacement,In stock,https://www.amazon.in/Juarez-Acoustic-Guitar-C...


# Q3. Write a python program to access the search bar and search button on images.google.com and scrape 10 images each for keywords ‘fruits’, ‘cars’ and ‘Machine Learning’, ‘Guitar’, ‘Cakes’.

In [84]:
import time
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By


# Navigate to images.google.com
driver.get("https://images.google.com/")

In [86]:
# Define the list of keywords
keywords = ['fruits', 'cars', 'Machine Learning', 'Guitar', 'Cakes']

for keyword in keywords:
    try:
        # Find the search bar
        search_bar = driver.find_element(By.XPATH, '//textarea[@class="gLFyf"]')

        # Clear the search bar and enter the keyword
        search_bar.clear()
        search_bar.send_keys(keyword)

        # Find the search button and click it
        search_button = driver.find_element(By.XPATH, '//div[@class="zgAlFc"]')
        search_button.click()

        # Wait for the search results to load
        time.sleep(3)

        # Scroll the webpage multiple times to load more images
        for _ in range(20):
            driver.execute_script("window.scrollBy(0,1000)")
            time.sleep(1)

        # Find the image elements
        images = driver.find_elements(By.XPATH, '//img[@class="rg_i Q4LuWd"]')

        # Scrape the image URLs
        img_urls = []
        for image in images:
            source = image.get_attribute('src')
            if source is not None and source[0:4] == 'http':
                img_urls.append(source)

        # Download the images
        for i in range(min(10, len(img_urls))):
            print("Downloading {0} of {1} images".format(i+1, min(10, len(img_urls))))
            response = requests.get(img_urls[i])
            with open(f"{keyword}_image_{i+1}.jpg", "wb") as file:
                file.write(response.content)

        # Wait before proceeding to the next keyword
        time.sleep(2)

    except Exception as e:
        print("An error occurred:", e)


An error occurred: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//textarea[@class="gLFyf"]"}
  (Session info: chrome=114.0.5735.134)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x0099A813+48355]
	(No symbol) [0x0092C4B1]
	(No symbol) [0x00835358]
	(No symbol) [0x008609A5]
	(No symbol) [0x00860B3B]
	(No symbol) [0x0088E232]
	(No symbol) [0x0087A784]
	(No symbol) [0x0088C922]
	(No symbol) [0x0087A536]
	(No symbol) [0x008582DC]
	(No symbol) [0x008593DD]
	GetHandleVerifier [0x00BFAABD+2539405]
	GetHandleVerifier [0x00C3A78F+2800735]
	GetHandleVerifier [0x00C3456C+2775612]
	GetHandleVerifier [0x00A251E0+616112]
	(No symbol) [0x00935F8C]
	(No symbol) [0x00932328]
	(No symbol) [0x0093240B]
	(No symbol) [0x00924FF7]
	BaseThreadInitThunk [0x77C700C9+25]
	RtlGetAppContainerNamedObjectPath [0x77DE7B4E+286]
	RtlGetAppContainerNamedObjectPath [0x77DE7B1E+238]

An error occurred: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//t

In [87]:
from selenium import webdriver
import time
import urllib.request

# Set up the Selenium driver
driver = webdriver.Chrome('/path/to/chromedriver')

# Define the list of keywords
keywords = ['fruits', 'cars', 'Machine Learning', 'Guitar', 'Cakes']

# Loop through each keyword
for keyword in keywords:
    # Access images.google.com
    driver.get('https://images.google.com')

    # Find the search bar and enter the keyword
    search_bar = driver.find_element_by_xpath('//input[@class="gLFyf"]')
    search_bar.send_keys(keyword)

    # Find the search button and click it
    search_button = driver.find_element_by_xpath('//button[@class="Tg7LZd"]')
    search_button.click()

    # Scroll down to load more images
    for _ in range(3):
        driver.execute_script('window.scrollBy(0, document.body.scrollHeight);')
        time.sleep(2)

    # Find all the image elements
    image_elements = driver.find_elements_by_xpath('//img[@class="rg_i Q4LuWd"]')

    # Download the first 10 images
    for i in range(10):
        try:
            image_url = image_elements[i].get_attribute('src')
            urllib.request.urlretrieve(image_url, f'{keyword}_{i+1}.jpg')
            print(f'Downloaded {i+1} of 10 images for {keyword}')
        except:
            print(f'Error downloading image {i+1} for {keyword}')




AttributeError: 'WebDriver' object has no attribute 'find_element_by_xpath'

# Q4. Write a python program to search for a smartphone(e.g.: Oneplus Nord, pixel 4A, etc.) on www.flipkart.com and scrape following details for all the search results displayed on 1st page. 
Details to be scraped: “Brand Name”, “Smartphone name”, “Colour”, “RAM”, “Storage(ROM)”, “Primary Camera”,
“Secondary Camera”, “Display Size”, “Battery Capacity”, “Price”, “Product URL”. Incase if any of the details is missing then replace it by “- “. Save your results in a dataframe and CSV.

In [120]:
driver.get("https://www.flipkart.com/")

In [121]:
search_input = driver.find_element(By.XPATH, '//input[@class="_3704LK"]')
search_input.send_keys('pixel 4A')
search_button = driver.find_element(By.XPATH, '//button[@class="L0Z3Pu"]')
search_button.click()
time.sleep(3)

In [122]:
# Scrap all product URLs
product_urls = []
start = 0
end = 1
for page in range(start, end):  
    urls = driver.find_elements(By.XPATH, '//a[@class="_1fQZEK"]')
    for url in urls:
        product_urls.append(url.get_attribute("href"))
    time.sleep(2)

In [123]:
len(product_urls)

24

In [125]:
# Define lists to store the product details
Brand = []
Name = []
Color = []
RAM = []
ROM = []
Primary_Camera = []
Secondary_Camera = []
Display_Size = []
Battery_Capacity = []
Price = []
product_details = []

# Loop through each product URL and scrape the details
for url in product_urls:
    driver.get(url)
    time.sleep(2)
    
    # Find the read more and click it
    read_more = driver.find_element(By.XPATH, '//button[@class="_2KpZ6l _1FH0tX"]')
    read_more.click()
        
    # Scrape the brand name
    try:
        brand = driver.find_element(By.XPATH, '//*[@id="container"]/div/div[3]/div[1]/div[2]/div[9]/div[5]/div/div[2]/div[1]/div[1]/table/tbody/tr[3]/td[2]/ul/li').text
    except NoSuchElementException:
        brand = '-'
    Brand.append(brand)

    # Scrape the product name
    try:
        name = driver.find_element(By.XPATH, '//*[@id="container"]/div/div[3]/div[1]/div[2]/div[2]/div/div[1]/h1/span').text
    except NoSuchElementException:
        name = '-'
    Name.append(name)

    # Scrape the product color
    try:
        color = driver.find_element(By.XPATH, '//*[@id="container"]/div/div[3]/div[1]/div[2]/div[9]/div[5]/div/div[2]/div[1]/div[1]/table/tbody/tr[4]/td[2]/ul/li').text
    except NoSuchElementException:
        color = '-'
    Color.append(color)

    # Scrape the product RAM
    try:
        ram = driver.find_element(By.XPATH, '//*[@id="container"]/div/div[3]/div[1]/div[2]/div[9]/div[4]/div/div[2]/div[1]/div[4]/table/tbody/tr[2]/td[2]/ul/li').text
    except NoSuchElementException:
        ram = '-'
    RAM.append(ram)

    # Scrape the product ROM
    try:
        rom = driver.find_element(By.XPATH, '//*[@id="container"]/div/div[3]/div[1]/div[2]/div[9]/div[4]/div/div[2]/div[1]/div[4]/table/tbody/tr[1]/td[2]/ul/li').text
    except NoSuchElementException:
        rom = '-'
    ROM.append(rom)

    # Scrape the product primary camera
    try:
        primary_camera = driver.find_element(By.XPATH, '//*[@id="container"]/div/div[3]/div[1]/div[2]/div[9]/div[4]/div/div[2]/div[1]/div[5]/table/tbody/tr[2]/td[2]/ul/li').text
    except NoSuchElementException:
        primary_camera = '-'
    Primary_Camera.append(primary_camera)

    # Scrape the product secondary camera
    try:
        secondary_camera = driver.find_element(By.XPATH, '//*[@id="container"]/div/div[3]/div[1]/div[2]/div[9]/div[5]/div/div[2]/div[1]/div[5]/table/tbody/tr[5]/td[2]/ul/li').text
    except NoSuchElementException:
        secondary_camera = '-'
    Secondary_Camera.append(secondary_camera)

    # Scrape the product display size
    try:
        display_size = driver.find_element(By.XPATH, '//*[@id="container"]/div/div[3]/div[1]/div[2]/div[9]/div[5]/div/div[2]/div[1]/div[2]/table/tbody/tr[1]/td[2]/ul/li').text
    except NoSuchElementException:
        display_size = '-'
    Display_Size.append(display_size)

    # Scrape the product battery capacity
    try:
        battery_capacity = driver.find_element(By.XPATH, '//*[@id="container"]/div/div[3]/div[1]/div[2]/div[8]/div/div/div[2]/ul/li[4]').text
    except NoSuchElementException:
        battery_capacity = '-'
    Battery_Capacity.append(battery_capacity)

    # Scrape the price
    try:
        price = driver.find_element(By.XPATH, '//*[@id="container"]/div/div[3]/div[1]/div[2]/div[2]/div/div[4]/div[1]/div/div[1]').text
    except NoSuchElementException:
        price = '-'
    Price.append(price)

    # Append the product details to the list
    product_details.append([brand, name, color, ram, rom, primary_camera, secondary_camera, display_size, battery_capacity, price, url])

# Create a DataFrame with the scraped product details
df = pd.DataFrame(product_details, columns=["Brand Name", "Smart Phone Name", "Color", "RAM", "Storage '(ROM)'", "Primary Camera", "Secondary Camera", "Display Size", "Battery Capacity", "Price", "Product URL"])

# Display the full DataFrame without truncation
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
display(df)

# Save the DataFrame as a CSV file
df.to_csv("product_details.csv", index=False)


,Brand Name,Smart Phone Name,Color,RAM,Storage '(ROM)',Primary Camera,Secondary Camera,Display Size,Battery Capacity,Price,Product URL
0,-,"realme C33 2023 (Aqua Blue, 128 GB) (4 GB RAM)",-,4 GB,128 GB,50MP + 0.3MP,-,-,5000 mAh Lithium Ion Battery,"₹10,499",https://www.flipkart.com/realme-c33-2023-aqua-...
1,-,"realme C33 2023 (Night Sea, 128 GB) (4 GB RAM)",-,4 GB,128 GB,50MP + 0.3MP,-,-,5000 mAh Lithium Ion Battery,"₹10,499",https://www.flipkart.com/realme-c33-2023-night...
2,C30,"realme C30 (Bamboo Green, 32 GB) (3 GB RAM)",Bamboo Green,-,-,-,5MP Front Camera,16.51 cm (6.5 inch),5000 mAh Lithium Ion Battery,"₹6,499",https://www.flipkart.com/realme-c30-bamboo-gre...
3,C30,"realme C30 (Denim Black, 32 GB) (2 GB RAM)",Denim Black,-,-,-,5MP Front Camera,16.51 cm (6.5 inch),5000 mAh Lithium Ion Battery,"₹5,999",https://www.flipkart.com/realme-c30-denim-blac...
4,C30,"realme C30 (Lake Blue, 32 GB) (3 GB RAM)",Lake Blue,-,-,-,5MP Front Camera,16.51 cm (6.5 inch),5000 mAh Lithium Ion Battery,"₹6,499",https://www.flipkart.com/realme-c30-lake-blue-...
5,C30,"realme C30 (Denim Black, 32 GB) (3 GB RAM)",Denim Black,-,-,-,5MP Front Camera,16.51 cm (6.5 inch),5000 mAh Lithium Ion Battery,"₹6,499",https://www.flipkart.com/realme-c30-denim-blac...
6,10 Pro 5G,"realme 10 Pro 5G (Dark Matter, 128 GB) (8 GB ...",Dark Matter,-,-,-,16MP Front Camera,17.07 cm (6.72 inch),5000 mAh Battery,"₹19,999",https://www.flipkart.com/realme-10-pro-5g-dark...
7,10 Pro 5G,"realme 10 Pro 5G (Hyperspace, 128 GB) (6 GB RAM)",Hyperspace,-,-,-,16MP Front Camera,17.07 cm (6.72 inch),5000 mAh Battery,"₹18,999",https://www.flipkart.com/realme-10-pro-5g-hype...
8,10 Pro 5G,"realme 10 Pro 5G (Nebula Blue, 128 GB) (8 GB ...",Nebula Blue,-,-,-,16MP Front Camera,17.07 cm (6.72 inch),5000 mAh Battery,"₹19,999",https://www.flipkart.com/realme-10-pro-5g-nebu...
9,-,"realme 10 Pro 5G (Dark Matter, 128 GB) (6 GB ...",-,-,-,-,-,-,-,"₹18,999",https://www.flipkart.com/realme-10-pro-5g-dark...


# Q5. Write a program to scrap geospatial coordinates (latitude, longitude) of a city searched on google maps

In [188]:
def get_coordinates(city_name):
  """Gets the geospatial coordinates of a city searched on Google Maps.

  Args:
    city_name: The name of the city to search for.

  Returns:
    A tuple of latitude and longitude coordinates.
  """

  driver = webdriver.Chrome()
  driver.get("https://www.google.co.uk/maps/")
  sleep(2)

  # Find the search input field and enter the city name
  search_input = driver.find_element(By.XPATH, '//*[@id="searchboxinput"]')
  search_input.send_keys(city_name)

  # Click the search button
  search_button = driver.find_element(By.XPATH, '//*[@id="searchbox-searchbutton"]')
  search_button.click()

  sleep(5)

  # Extract the latitude and longitude coordinates from the URL.
  url_string = driver.current_url
  print("URL Extracted:", url_string)
  lat_lng = re.findall(r'@(.*)data', url_string)
  latitude = lat_lng[0].split(",")[0]
  longitude = lat_lng[0].split(",")[1]

  return latitude, longitude

if __name__ == "__main__":
  city_name = input("Enter the city name: ")
  latitude, longitude = get_coordinates(city_name)
  print(f"The geospatial coordinates of {city_name} are {latitude}, {longitude}")


Enter the city name: Sheffield
URL Extracted: https://www.google.co.uk/maps/place/Sheffield/@53.3956741,-1.8283974,10z/data=!3m1!4b1!4m6!3m5!1s0x48790aa9fae8be15:0x3e2827f5af06b078!8m2!3d53.381129!4d-1.470085!16zL20vMG03NWc?entry=ttu
The geospatial coordinates of Sheffield are 53.3956741, -1.8283974


# Q6. Write a program to scrap all the available details of best gaming laptops from digit.in.

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By

def scrape_gaming_laptops():
    # Set up the Chrome driver
    driver = webdriver.Chrome()

    # Navigate to the URL
    url = 'https://www.digit.in/top-products/best-gaming-laptops-40.html'
    driver.get(url)

    # Find all the individual laptop entries
    laptop_entries = driver.find_elements(By.CLASS_NAME, 'TopNumbeHeading')

    # Iterate over each laptop entry and extract the details
    for entry in laptop_entries:
        # Extract the laptop name
        laptop_name = entry.find_element(By.CLASS_NAME, 'TopNumbeHeading').text.strip()

        # Extract the laptop specifications
        specs = entry.find_element(By.CLASS_NAME, 'specification').text.strip()

        # Extract the laptop price
        laptop_price = entry.find_element(By.CLASS_NAME, 'smprice').text.strip()

        # Print the details of each laptop
        print('Laptop:', laptop_name)
        print('Specifications:', specs)
        print('Price:', laptop_price)
        print('---')

    # Remember to close the browser after scraping
    driver.close()

# Run the function to scrape gaming laptop details
scrape_gaming_laptops()


# Q7. Write a python program to scrape the details for all billionaires from www.forbes.com. Details to be scrapped: “Rank”, “Name”, “Net worth”, “Age”, “Citizenship”, “Source”, “Industry”.

# Q8. Write a program to extract at least 500 Comments, Comment upvote and time when comment was posted from any YouTube Video.

# Q9. Write a python program to scrape a data for all available Hostels from https://www.hostelworld.com/ in “London” location. You have to scrape hostel name, distance from city centre, ratings, total reviews, overall reviews, privates from price, dorms from price, facilities and property description.